In [6]:
import os
import sys
module_path = os.path.abspath(os.path.join('../trading-bot'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
import pandas as pd
import numpy as np

In [8]:
df_raw = pd.read_csv("../resources/Bitstamp_BTCUSD_2021_minute.csv", skiprows=1)

In [9]:
df = df_raw.copy()
df.drop(['date', 'symbol'], axis=1, inplace=True)
df.columns = ["unixtimestamp_millis", "open", "high", "low", "close", "volume_btc", "volume_usd"]
df["unixtimestamp_millis"] = df["unixtimestamp_millis"] * 1000
df.insert(loc=0, column='datetime', value=pd.to_datetime(df['unixtimestamp_millis'], unit='ms'))
df = df.iloc[::-1]
df.set_index("datetime", inplace=True)
df = df.resample("5T").agg({"unixtimestamp_millis": "first",
                                        "open":  "first",
                                        "high":  "max",
                                        "low":   "min",
                                        "close": "last",
                                        "volume_btc": "sum",
                                        "volume_usd": "sum"
                                    })
df.reset_index(inplace=True)
df["datetime"] = df["datetime"].shift(-1)
df["unixtimestamp_millis"] = df["unixtimestamp_millis"].shift(-1)
df = df[:-1]
df = df.astype({"unixtimestamp_millis": np.int64, "volume_btc": np.int64, "volume_usd": np.int64})

In [10]:
df

,datetime,unixtimestamp_millis,open,high,low,close,volume_btc,volume_usd
0,2021-01-01 00:05:00,1609459500000,29007.31,29086.90,29007.31,29045.19,21,617118
1,2021-01-01 00:10:00,1609459800000,29021.86,29025.68,28910.15,28937.66,8,259818
2,2021-01-01 00:15:00,1609460100000,28932.99,28945.01,28774.64,28834.74,40,1166077
3,2021-01-01 00:20:00,1609460400000,28812.07,28916.52,28800.92,28916.52,10,303865
4,2021-01-01 00:25:00,1609460700000,28893.72,28919.39,28802.97,28907.28,16,465005
...,...,...,...,...,...,...,...,...
99492,2021-12-12 11:05:00,1639307100000,48964.56,49036.99,48933.56,48933.56,0,23694
99493,2021-12-12 11:10:00,1639307400000,48890.17,48957.32,48866.09,48937.44,2,122581
99494,2021-12-12 11:15:00,1639307700000,48930.55,49017.38,48930.27,48944.30,0,40578
99495,2021-12-12 11:20:00,1639308000000,48958.00,49030.27,48950.01,49010.31,1,57165


In [11]:
df.dtypes

datetime                datetime64[ns]
unixtimestamp_millis             int64
open                           float64
high                           float64
low                            float64
close                          float64
volume_btc                       int64
volume_usd                       int64
dtype: object

In [20]:
df_bt_res = pd.read_csv("../resources/reports/report1.csv", sep=",")
df_bt_res.insert(loc=0, column='datetime', value=pd.to_datetime(df_bt_res['timestamp_millis'], unit='ms'))
df_bt_res.head()

,datetime,timestamp_millis,btc,overall
0,2021-01-01 09:45:00,1609494300000,0.000341,50000.000000
1,2021-01-01 10:20:00,1609496400000,0.000341,49999.996049
2,2021-01-01 10:55:00,1609498500000,0.000341,49999.998986
3,2021-01-01 11:30:00,1609500600000,0.000340,50000.064868
4,2021-01-01 12:05:00,1609502700000,0.000340,50000.030000
